cover story

# Section 0: Import Packages

In [ ]:
#to deal with pandas dataframe and import data
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, roc_auc_score,roc_curve
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

# Section 1: Data Pre-processing

In [ ]:
# read dataset
url = 'https://raw.githubusercontent.com/chaiwencw/Heart-Disease-Prediction-Using-Machine-Learning/main/heart-disease-data.csv'
raw_data = pd.read_csv(url)

In [ ]:
# print center-aligned dataframe
def center_aligned(df):
  dfStyler = df.style.set_properties(**{'text-align': 'center'})
  return dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
  
center_aligned(raw_data.head())

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.000000,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.100000,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.600000,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.000000,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.900000,1,3,2,0


Click to View ▶ [Variable Description](https://drive.google.com/uc?id=12Fp0Q1zQioOgoDwC1Si_VMB6sRPVckTC)


In [ ]:
# rename columns
raw_data.columns = ['Age','Gender', 'Chest Pain Type','Rest Blood Pressure','Serum Cholestrol',
                    'Fasting Blood Sugar', 'Resting ECG','Max Heart Rate', 'Exercise Induced Angina', 'Old Peak',
                    'Slope of the Peak','No. of Major Vessels','Thalassemia', 'Diagnosis of Heart Disease']

center_aligned(raw_data.head())

,Age,Gender,Chest Pain Type,Rest Blood Pressure,Serum Cholestrol,Fasting Blood Sugar,Resting ECG,Max Heart Rate,Exercise Induced Angina,Old Peak,Slope of the Peak,No. of Major Vessels,Thalassemia,Diagnosis of Heart Disease
0,52,1,0,125,212,0,1,168,0,1.000000,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.100000,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.600000,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.000000,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.900000,1,3,2,0


In [ ]:
# determine no. of rows & columns
raw_data.shape

(1025, 14)

In [ ]:
# check number of unique value in each column
raw_data.nunique()

Age                            41
Gender                          2
Chest Pain Type                 4
Rest Blood Pressure            49
Serum Cholestrol              152
Fasting Blood Sugar             2
Resting ECG                     3
Max Heart Rate                 91
Exercise Induced Angina         2
Old Peak                       40
Slope of the Peak               3
No. of Major Vessels            5
Thalassemia                     4
Diagnosis of Heart Disease      2
dtype: int64

The above result shows the number of unique elements in **No. of Major Vessels** and **Thalassemia** are 5 and 4, respectively. But based on the [Variable Description](https://drive.google.com/uc?id=12Fp0Q1zQioOgoDwC1Si_VMB6sRPVckTC), they should be 4 (0, 1, 2, 3) and 3 (0, 1, 2), accordingly. So, there may exist missing data in these columns.



Handling Missing Values: No. of Major Vessels and Thalassemia

---



In [ ]:
# check unique element in columns with missing data
def unique_element(md):
  for col in md:
    print(f'The unique elements in {col} is {md[col].unique()}')

unique_element(raw_data[['No. of Major Vessels','Thalassemia']])

The unique elements in No. of Major Vessels is [2 0 1 3 4]
The unique elements in Thalassemia is [3 2 1 0]


In [ ]:
# change the missing values to nan
raw_data.loc[(raw_data['No. of Major Vessels'] == 4),'No. of Major Vessels'] = None
raw_data.loc[(raw_data['Thalassemia'] == 3),'Thalassemia'] = None
unique_element(raw_data[['No. of Major Vessels','Thalassemia']])

The unique elements in No. of Major Vessels is [ 2.  0.  1.  3. nan]
The unique elements in Thalassemia is [nan  2.  1.  0.]


In [ ]:
# count missing data
vessel_null = raw_data['No. of Major Vessels'].isnull().sum()
print(f' The total missing values in No. of Major Vessels is {vessel_null}')
thal_null = raw_data['Thalassemia'].isnull().sum()
print(f' The total missing values in Thalassemia is {thal_null}')

 The total missing values in No. of Major Vessels is 18
 The total missing values in Thalassemia is 410


In [ ]:
# missing values imputation using KNNImputer
def knn_imputer(col_with_null):
  imputer = KNNImputer(n_neighbors=2)
  result = np.rint(imputer.fit_transform(col_with_null))
  return result
raw_data['No. of Major Vessels'] = knn_imputer(raw_data[['No. of Major Vessels']])
raw_data['Thalassemia'] = knn_imputer(raw_data[['Thalassemia']])

In [ ]:
# fig, ax = plt.subplots(2,2,figsize=(20,8))
# fig = raw_data['No. of Major Vessels'].fillna('NULL').value_counts().plot.bar(rot=0,ax=ax[0][0])
# fig.set(title =f'Count Plot of No. of Major Vessels Before Imputation')
# fig = raw_data['Thalassemia'].fillna('NULL').value_counts().plot.bar(rot=0,ax=ax[1][0])
# fig.set(title =f'Count Plot of Thalassemia Before Imputation')
# raw_data['No. of Major Vessels'] = knn_imputer(raw_data[['No. of Major Vessels']])
# raw_data['Thalassemia'] = knn_imputer(raw_data[['Thalassemia']])
# fig = raw_data['No. of Major Vessels'].value_counts().plot.bar(rot=0,ax=ax[0][1])
# fig.set(title =f'Count Plot of No. of Major Vessels After Imputation')
# fig = raw_data['Thalassemia'].value_counts().plot.bar(rot=0,ax=ax[1][1])
# fig.set(title =f'Count Plot of Thalassemia After Imputation')
# plt.show()



---


Decode the encoded categorical columns for readability.

In [ ]:
# create attribute dictionary
attr_dict = { 
    'Gender': {
        0: 'female',
        1: 'male'
      },
    'Chest Pain Type': {
        0: 'typical angina', 
        1: 'atypical angina', 
        2: 'non-anginal pain',
        3: 'asymptomatic'
    },
    'Fasting Blood Sugar': {0: 'false', 1: 'true'},
    'Rest ECG': {
        0: 'normal',
        1: 'abnormal',
        2: 'hyper'
    }, 
    'Exercise Induced Angina': {0: 'no', 1: 'yes'},
    'Slope of the Peak': {
        0: 'up',
        1: 'flat',
        2: 'down'
    },
    'Thalassemia': { #the current dataset: 0 = null
        0: 'normal',
        1: 'fixed defect',
        2: 'reversible defect'
    }, 
    'Diagnosis of Heart Disease': {0: 'no', 1: 'yes'}
}


# map attribute dictionary to the categorical columns (decode numeric values to string)
decoded_data = pd.DataFrame()
for col in raw_data.columns:
  if col in attr_dict:
    decoded_data[col] = raw_data[col].map(attr_dict[col].get)
  else:
    decoded_data[col] = raw_data[col]

center_aligned(decoded_data.head())


,Age,Gender,Chest Pain Type,Rest Blood Pressure,Serum Cholestrol,Fasting Blood Sugar,Resting ECG,Max Heart Rate,Exercise Induced Angina,Old Peak,Slope of the Peak,No. of Major Vessels,Thalassemia,Diagnosis of Heart Disease
0,52,male,typical angina,125,212,false,1,168,no,1.000000,down,2.000000,reversible defect,no
1,53,male,typical angina,140,203,true,0,155,yes,3.100000,up,0.000000,reversible defect,no
2,70,male,typical angina,145,174,false,1,125,yes,2.600000,up,0.000000,reversible defect,no
3,61,male,typical angina,148,203,false,1,161,no,0.000000,down,1.000000,reversible defect,no
4,62,female,typical angina,138,294,true,1,106,no,1.900000,flat,3.000000,reversible defect,no


In [ ]:
# quick view basic information of the dataset
decoded_data.info()

# Section 2: Exploratory Data Analysis (EDA)

In [ ]:
cat_features = decoded_data.drop('Diagnosis of Heart Disease', axis=1).select_dtypes(include='object')
# num_features consists of all numerical columns only
num_features = decoded_data.select_dtypes(exclude = 'object')
target=decoded_data[['Diagnosis of Heart Disease']]

A. Categorical Data Analysis:


1.   Gender



In [ ]:
# visualize frequency of individuals based on Gender using bar plot
fig = sns.countplot(x=decoded_data['Gender'])
fig.set(xlabel='Gender',
        ylabel='Number of Individuals',
        title="Bar Plot of Gender")
# to display count values on the top of bar
for p in fig.patches:
   fig.annotate('{:.1f}'.format(p.get_height()), 
                (p.get_x()+p.get_width()/2, 
                 p.get_height()), ha ='center') #adjust position to center
plt.show()

2. Chest Pain Type

In [ ]:
fig = sns.countplot(x=decoded_data['Fasting Blood Sugar'])
fig.set(xlabel='Fasting Blood Sugar ',
        ylabel='Number of Individuals',
        title="Bar Plot of Fasting Blood Sugar ")
# to display count values on the top of bar
for p in fig.patches:
   fig.annotate('{:.1f}'.format(p.get_height()), 
                (p.get_x()+p.get_width()/2, 
                 p.get_height()), ha ='center') #adjust position to center
plt.show()

3. Exercise-Induced Angina

In [ ]:
decoded_data['Exercise Induced Angina'].value_counts(normalize = True).plot(kind='pie', 
                                                        autopct='%1.0f%%',
                                                        title='Exercise-Induced Angina?',
                                                        colormap = 'Pastel1')
plt.show()

4. Slope of the Peak

In [ ]:
decoded_data['Slope of the Peak'].value_counts(normalize = True).plot(kind='pie', 
                                                        autopct='%1.0f%%',
                                                        title='ST Slope',
                                                        colormap = 'Pastel1')
plt.show()

5. Thalassemia

In [ ]:
decoded_data['Thalassemia'].value_counts(normalize = True).plot(kind='pie', 
                                                        autopct='%1.0f%%',
                                                        colormap = 'Pastel1',)
                                                        
plt.show()


6. Diagnosis of Heart Disease (Target)

In [ ]:
# visualize frequency of individuals based on Gender using bar plot
fig = sns.countplot(x=target)
fig.set(xlabel='',
        ylabel='Number of Individuals',
        title="Bar Plot of Diagnosis of Heart Disease")
# to display count values on the top of bar
for p in fig.patches:
   fig.annotate('{:.1f}'.format(p.get_height()), 
                (p.get_x()+p.get_width()/2, 
                 p.get_height()), ha ='center') #adjust position to center
plt.show()

In [ ]:
# histogram

B. Numerical Data Analysis

Descriptive Statistics:

In [ ]:
round(num_features.describe().T,2)

In [ ]:
num_features.skew()

In [ ]:
num_features.kurtosis()

Distribution Plot:

In [ ]:
# distribution plot
fig = plt.figure(figsize=(12, 8))
num_features.hist(ax=fig.gca())
plt.show()

Remove Outliers:

In [ ]:
# Plot a boxplot for each numerical column 
fig, ax = plt.subplots(1, 7,figsize=(25,5))
for col in num_features:
  i = num_features.columns.get_loc(col)
  fig = sns.boxplot(x = num_features[col],ax=ax[i])
  fig.set(title =f'Boxplot of {col}')
plt.show()


In [ ]:
# Handling with Outliers
fig, ax = plt.subplots(1, 7,figsize=(25,5))
for col in num_features:
  q1 = num_features[col].quantile(0.25)
  q3 = num_features[col].quantile(0.75)
  IQR = q3 - q1
  lower_limit = q1 - 1.5 * IQR
  upper_limit = q3 + 1.5 * IQR

  k1=num_features[num_features[col]>upper_limit]
  k2=num_features[num_features[col]<lower_limit]
  data_clean = pd.DataFrame()
  
  if not k1[col].empty:
    print(f'Outliers on right side in {col} is \n{k1[col]}.\n')
    data_clean = num_features[num_features[col]<upper_limit]
  elif not k2[col].empty:
    print(f'Outliers on left side in {col} is \n{k2[col]}.\n')
    data_clean = num_features[num_features[col]>lower_limit]
  elif k1[col].empty and k2[col].empty:
    print(f'No outlier is found in {col}!\n')
    data_clean = num_features[(num_features[col]>lower_limit) & (num_features[col]<upper_limit)]
  i = data_clean.columns.get_loc(col)
  fig = sns.boxplot(x = data_clean[col],ax=ax[i])
plt.show()

In [ ]:
num_features

In [ ]:
#exclude outlier rows
cat_features = cat_features[cat_features.index.isin(num_features.index)]
target = target[target.index.isin(num_features.index)]
features = pd.concat([cat_features,num_features])

### Correlation

**Correlation Among Numeric Features**

In [ ]:
#obtain correlation matrix
corr_mat = features.corr().abs()

In [ ]:
#obtain upper triangle of the correlation matrix (no diagonal) 
upper_tri = corr_mat.where(np.triu(np.ones(corr_mat.shape),k=1).astype(bool)) #drop the columns (features) with the correlation coefficient > 0.6
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.6)] 
if len(to_drop) == 0:
  print('No highly correlated features are found!')
else:
  print(to_drop)

Since no features are highly correlated, no dropping is performed. We can visualize the features correlation by plotting heatmap.

In [ ]:
plt.figure(figsize = (15,8))
sns.heatmap(features.corr(),annot=True,cmap='YlGnBu')
plt.show()

Correlation between Categorical and Numeric Features 

Correlation between Features and Target

- Assume that good variables are highly correlated with the target.

In [ ]:
# reversed_thal_dict = {value: key for key, value in attr_dict['Thalassemia'].items()}
# thal_df = pd.DataFrame({'original-thalassemia': decoded_data['Thalassemia'].map(reversed_thal_dict)})
for col in cat_features:
  reversed_dict = {key for key, value in attr_dict[col].items()}
  cat_features[col]= cat_features[col].map(reversed_dict)

## Section 2.3: Data Splitting & Standardization

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.30, random_state=1)

In [ ]:
scalerX = PowerTransformer().fit(x_train)
scalerY = PowerTransformer().fit(y_train)
x_train = scalerX.transform(x_train)
y_train = scalerY.transform(y_train)
x_test = scalerX.transform(x_test)
y_test = scalerY.transform(y_test)

# Section 3: Analysis

## Section 3.1: Support Vector Machine (SVM)

In [ ]:
svm = SVC()
svm.fit(x_train, y_train)

## Section 3.2: k-Nearest Neighbors (kNN)

In [ ]:
knn = KNeighborsClassifierr()
knn.fit(x_train, y_train)

## Section 3.3: Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

In [ ]:
lr_pred = lr.predict(x_test)
pd.DataFrame(np.c_[y_test,lr_pred],columns=['Actual','Predicted']).head(7)

In [ ]:
print(" Test Accuracy score : ",round(lr.score(x_test,y_test)*100,2),"%")
print("Train Accuracy score : ",round(lr.score(x_train,y_train)*100,2),"%")
print("----------------------")
print("Classification Report")
print("----------------------")
con = confusion_matrix(lr_pred,y_test)
plt.figure(figsize=(5,3))
sns.heatmap(con, annot=True,cmap="YlGnBu",fmt='g')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
print(classification_report(y_test,lr_pred))

In [ ]:
auc = roc_auc_score(y_test,lr_pred)
auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,lr_pred)
plt.figure(figsize=(8,5))
plt.plot(fpr , tpr , color='red',label='ROC')
plt.plot([0,1],[0,1],color = 'darkblue',linestyle='--',label='ROC curve(area = %0.2f)'% auc)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver Operating Characterstics (ROC) curve')
plt.legend()
plt.show()

## Section 3.4: Random Forest Classification

In [ ]:
rfc = RandomForestClassifier(max_depth=4)
rfc.fit(x_train, y_train)

In [ ]:
rfc_pred = rfc.predict(x_test)
pd.DataFrame(np.c_[y_test,rfc_pred],columns=['Actual','Predicted']).head()

In [ ]:
print(" Test Accuracy score : ",round(rfc.score(x_test,y_test)*100,2),"%")
print("Train Accuracy score : ",round(rfc.score(x_train,y_train)*100,2),"%")
clf_con = confusion_matrix(y_test,rfc_pred)
print("----------------------")
print("Classification Report")
print("----------------------")
print(classification_report(y_test,rfc_pred))
plt.figure(figsize=(5,3))
sns.heatmap(clf_con, annot=True,cmap="YlGnBu",fmt='g')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

In [ ]:
auc = roc_auc_score(y_test,rfc_pred)
auc

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,rfc_pred)
plt.figure(figsize=(8,5))
plt.plot(fpr , tpr , color='red',label='ROC')
plt.plot([0,1],[0,1],color = 'darkblue',linestyle='--',label='ROC curve(area = %0.2f)'% auc)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Receiver Operating Characterstics (ROC) curve')
plt.legend()
plt.show()

## Section 3.5: Model Comparison

In [ ]:
data = { 'Models' : ['Logistic Regression','Random Forest'],
         'Test Accuracy' : [round(lr.score(x_test,y_test)*100,2), round(rfc.score(x_test,y_test)*100,2)],
         'Train Accuracy': [round(lr.score(x_train,y_train)*100,2),round(rfc.score(x_train,y_train)*100,2)],
       }

df = pd.DataFrame(data)
df.sort_values(by='Test Accuracy',ascending=False)

In [ ]:
plt.figure(figsize = (5,6))
sns.barplot(x='Models', y='Test Accuracy',data=df)
plt.show()

## Section 3.6: Data Testing

In [ ]:
data2.head(25)

In [ ]:
sample1 = [45,1,0,104,208,0,0,148,1,3.0,1,0,2]
if rfc.predict([sample1]) == 0:
    print('[SAFE] HEART DISEASE NOT DETECTED')
else:
    print(' [WARNING] HEART DISEASE DETECTED')

In [ ]:
sample2 = [45,1,0,104,208,0,0,148,1,3.0,1,0,2]
if lr.predict([sample2]) == 0:
    print('[SAFE] HEART DISEASE NOT DETECTED')
else:
    print(' [WARNING] HEART DISEASE DETECTED')

# Section 4: References



1. https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
2. DATA from UCI = https://archive.ics.uci.edu/ml/datasets/heart+disease
3. https://data.world/uci/heart-disease
4. var explanation & cover story: https://towardsdatascience.com/heart-disease-prediction-73468d630cfc
